In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely import Polygon

# Buildings

In [3]:
v = 'v10'


In [4]:
cluster_mapping = pd.read_parquet(f'/data/uscuni-ulce/processed_data/clusters/cluster_mapping_{v}.pq')
regions = gpd.read_parquet('/data/uscuni-ulce/regions/cadastre_regions_hull.parquet')

In [4]:
%%time

all_clusters = []

for region_id in regions.index:
    
    print(region_id)
    
    clusters = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/clusters/clusters_{region_id}_{v}.pq')
    clusters = clusters[['geometry', 'morph', 'final', 'final_without_noise']]
    clusters.index = str(region_id) + '_' + clusters.index.astype(str)
    
    clusters['initially_noise'] = clusters.final != clusters.final_without_noise
    
    for level in range(1,7):
        clusters[f'level_{level}_label'] = clusters.final_without_noise.map(cluster_mapping[level].to_dict())
    clusters[f'level_7_label'] = clusters.final_without_noise
    
    clusters = clusters[['geometry', 'morph', 'initially_noise', 'level_1_label', 'level_2_label', 'level_3_label', 'level_4_label', 'level_5_label', 'level_6_label', 'level_7_label']]
    
    all_clusters.append(clusters.to_crs(epsg=4236))

all_clusters = pd.concat(all_clusters)

4
10
132
134
286
313
400
523
765
801
832
913
960
1124
1154
1387
1478
1515
1605
1718
1736
1782
1970
1981
2096
2322
2350
2478
2514
2625
2728
2975
3039
3109
3150
3221
3250
3526
3610
3612
3701
3705
3752
3759
3981
4070
4214
4215
4235
4284
4356
4382
4723
4805
5096
5191
5246
5310
5408
5427
5662
5671
5766
5883
6254
6529
6560
6576
6741
6749
6811
6873
6996
7068
7094
7280
7485
7528
7534
7681
7688
7712
7727
7805
7914
7937
7963
8046
8216
8238
8256
8265
8345
8374
8396
8592
8707
8731
8757
8759
8813
9016
9064
9074
9150
9169
9194
9284
9824
9924
9954
9972
10019
10086
10095
10124
10179
10222
10263
10277
10455
10510
10511
10563
10579
10602
10666
10794
10847
10908
10926
10970
11002
11019
11057
11141
11210
11256
11261
11305
11309
11311
11318
11367
11444
11455
11471
11667
11678
11735
11757
11799
11877
11905
12027
12084
12100
12115
12154
12191
12381
12440
12483
12552
12667
12707
12755
12756
12844
12919
12965
13076
13137
13172
13191
13196
13229
13301
13395
13442
13482
13506
13553
13555
13614
13616
13655
13677


In [5]:
import gc
gc.collect()

0

In [6]:
all_clusters.head()

,geometry,morph,initially_noise,level_1_label,level_2_label,level_3_label,level_4_label,level_5_label,level_6_label,level_7_label
4_0,"POLYGON ((6.84318 51.04242, 6.84322 51.04245, ...",4_0_88,False,3,3,5,7,11,20,90
4_1,"POLYGON ((6.84996 51.04759, 6.85016 51.04784, ...",4_0_56,False,3,3,5,7,11,20,85
4_2,"POLYGON ((6.8481 51.05071, 6.84823 51.05079, 6...",4_0_97,False,3,3,5,7,11,20,88
4_3,"POLYGON ((6.84973 51.04307, 6.84981 51.04323, ...",4_0_-1,True,3,3,5,8,13,24,102
4_4,"POLYGON ((6.84778 51.04658, 6.84782 51.04662, ...",4_0_56,False,3,3,5,7,11,20,85


In [7]:
%%time
all_clusters.sort_values('geometry').to_parquet('/data/uscuni-ulce/data_product/buildings.parquet', index=True, geometry_encoding='geoarrow', write_covering_bbox=True, schema_version='1.1.0')

CPU times: user 2min 33s, sys: 22 s, total: 2min 56s
Wall time: 2min 55s


In [ ]:
# all_clusters.sort_values('geometry').to_file("/data/uscuni-ulce/data_product/buildings.fgb", engine="pyogrio")

# Linkage

In [6]:
linkage_matrix = np.load(f'/data/uscuni-ulce/processed_data/clusters/complete_linkage_10_{v}.npy')


In [7]:
np.save(f'/data/uscuni-ulce/data_product/ward_linkage_10.npy', linkage_matrix)

# Data

primary chars, morphotope labels, morphotope variables

In [14]:
regions = gpd.read_parquet('/data/uscuni-ulce/regions/cadastre_regions_hull.parquet')

In [8]:
# !cp /data/uscuni-ulce/processed_data/chars/primary_chars_*.parquet /data/uscuni-ulce/data_product/data/

In [10]:
model_params = '_post_processing_v1'
# region_id = 69333

In [19]:
%%time

all_morphs = []

for region_id in regions.index:
    print(region_id)
    morphotopes = pd.read_parquet(f'/data/uscuni-ulce/processed_data/morphotopes/tessellation_labels_morphotopes_{region_id}{model_params}.pq').morphotope_label
    morphotopes.index = str(region_id) + '_' + morphotopes.index.astype(str)
    all_morphs.append(morphotopes)

all_morphs = pd.concat(all_morphs)

4
10
132
134
286
313
400
523
765
801
832
913
960
1124
1154
1387
1478
1515
1605
1718
1736
1782
1970
1981
2096
2322
2350
2478
2514
2625
2728
2975
3039
3109
3150
3221
3250
3526
3610
3612
3701
3705
3752
3759
3981
4070
4214
4215
4235
4284
4356
4382
4723
4805
5096
5191
5246
5310
5408
5427
5662
5671
5766
5883
6254
6529
6560
6576
6741
6749
6811
6873
6996
7068
7094
7280
7485
7528
7534
7681
7688
7712
7727
7805
7914
7937
7963
8046
8216
8238
8256
8265
8345
8374
8396
8592
8707
8731
8757
8759
8813
9016
9064
9074
9150
9169
9194
9284
9824
9924
9954
9972
10019
10086
10095
10124
10179
10222
10263
10277
10455
10510
10511
10563
10579
10602
10666
10794
10847
10908
10926
10970
11002
11019
11057
11141
11210
11256
11261
11305
11309
11311
11318
11367
11444
11455
11471
11667
11678
11735
11757
11799
11877
11905
12027
12084
12100
12115
12154
12191
12381
12440
12483
12552
12667
12707
12755
12756
12844
12919
12965
13076
13137
13172
13191
13196
13229
13301
13395
13442
13482
13506
13553
13555
13614
13616
13655
13677


In [22]:
all_morphs = all_morphs.to_frame().reset_index()
all_morphs.columns = ['building_index', 'morphotope_label']
all_morphs.to_parquet('/data/uscuni-ulce/data_product/morphotope_labels.parquet')

### Morph chars

In [6]:
morphotopes_dir = '/data/uscuni-ulce/processed_data/morphotopes/'

def read_morphotopes_chars():
    region_hulls = gpd.read_parquet('/data/uscuni-ulce/regions/cadastre_regions_hull.parquet')
    data = []
    for region_id, _ in region_hulls.iterrows():
        extra_chars = pd.read_parquet(f'{morphotopes_dir}morph_chars_{region_id}.pq')
        data.append(extra_chars)
    
    data = pd.concat(data)
    return data 

In [7]:
%%time


morph_data = read_morphotopes_chars()

CPU times: user 1.16 s, sys: 207 ms, total: 1.36 s
Wall time: 1.04 s


In [9]:
morph_data.to_parquet('/data/uscuni-ulce/data_product/morphotope_characters.parquet')